In [17]:
########## Import ##########

import os
from simplified_scrapy import SimplifiedDoc, utils
import pandas as pd

import traci
import sumolib
from tqdm import tqdm
import random
import time
import matplotlib.pyplot as plt

import CH
import PredictPos
import ModifyFile
import safety_iov

In [2]:
########## Initialisation ##########

listTime = []
listNbVeh = []
listTotLenVeh = []

f_memory = open("memorybasics.csv", "w")
f_memory.write('Information,x,y' + "\n")
f_memory.close()

f_time = open("time.csv", "w")
f_time.write('Nb car,time' + "\n")
f_time.close()

f_rsu = open("rsu.csv", "w")
f_rsu.write('previous,future' + "\n")
f_rsu.close()

sumoBinary = "/Users/remi/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary,'-c','data/one_run/m/osm.sumocfg','-n', 'data/one_run/m/osm.net.xml']

time_start = time.time()
traci.start(sumoCmd)

########## Position of Traffic Light ##########

net = sumolib.net.readNet('data/one_run/m/osm.net.xml')

node_ids = [node.getID() for node in net.getNodes()]
traffic_pos = []

for node in node_ids:
    if (net.getNode(node).getType()) == 'traffic_light':
        traffic_pos.append(net.getNode(node).getCoord())

########## Selection of the lanes and edges ##########

TotalLane = traci.lane.getIDList()
TotalListLane = list(TotalLane)
ListRemove = []

TotalEdge = traci.edge.getIDList()
TotalBigEdge = list(TotalEdge)

for edge in TotalEdge:
    if(traci.edge.getLaneNumber(edge) < 2):
        TotalBigEdge.remove(edge)

allCar = []

for edge in TotalBigEdge:
    for lane in TotalLane:
        if(lane[0:10] in edge):
            if lane in TotalListLane:
                TotalListLane.remove(lane)

TotalLane = tuple(TotalListLane)

TotalLane = tuple(TotalListLane)
for lane in TotalLane:
    if(traci.lane.getMaxSpeed(lane)< 8.3):   #8.34 = 30 km/h
        TotalListLane.remove(lane)

########## Some informations ##########

print('number of TL: ', len(traffic_pos))
print('edge avant: ', len(TotalEdge), 'edge après: ', len(TotalBigEdge))
print('lane avant: ', len(TotalLane), 'lane après: ', len(TotalListLane))

 Retrying in 1 seconds
number of TL:  6
edge avant:  3312 edge après:  82
lane avant:  2729 lane après:  365


In [3]:
########## Launch of the simulation ##########

count = 2000 #960240
i = 0
while traci.simulation.getTime() <= count:

########## Simulation Step ##########

    traci.simulationStep()
    current_time = traci.simulation.getTime()

    action= i%9

########## List Vehicle ##########

    listVehID = []

    for edge in TotalBigEdge:
        listVeh = traci.edge.getLastStepVehicleIDs(edge)
        listVehID = listVehID + list(listVeh)
    for lane in TotalListLane:
        listVeh = traci.lane.getLastStepVehicleIDs(lane)
        listVehID = listVehID + list(listVeh)
    
    listVehID.sort()
    listVehID = list(set(listVehID))

########## CH ##########

    if i == 10:
        listNeighbor = CH.detectNeighbourg(listVehID)
        [listRole, listIDassigned, listRank]= CH.roleCar(listVehID,listNeighbor)
        qty = CH.assignColor(listIDassigned, listRole)
    if i > 10:
        time0 = time.time()
        listNeighbor = CH.detectNeighbourg(listVehID)
        [ListRole, listIDassigned, ListRank] = CH.maintainCH(listVehID, listIDassigned, listRank, listRole, listNeighbor)
        time1 = time.time()
        qty = CH.assignColor(listIDassigned, listRole)

        listTotLenVeh.append(len(listVehID))

        f_time = open("time.csv", "a")
        f_veh = len(listVehID)
        f_duration = time1 - time0
        f_time.write(str(f_veh) + "," + str(f_duration) + "\n")
        f_time.close

########## Prediction position ##########

    memory_time = '\ntime: %d \n' %traci.simulation.getTime()
    f_memory = open("memorybasics.csv", "a")
    f_memory.write(memory_time + "\n")

    for vehicle in listVehID:
        if vehicle in allCar:
            pass
        else:
            allCar.append(vehicle)
        position = traci.vehicle.getPosition(vehicle)
        futurPos = PredictPos.FuturPos(vehicle)
        memory_vehi = 'vehicle: %s' %vehicle
        memory_pos = 'Current Position:, %f,%f' %(position[0],position[1])
        memory_fut = 'Predict Position:, %f,%f' %(futurPos[0],futurPos[1])
        f_memory.write(memory_vehi + "\n" + memory_pos + "\n" + memory_fut + "\n")

    f_memory.close()

    i = i + 1 

########## End simulation ##########

time_end = time.time()
print ("traci time elapsed: {}".format(time_end-time_start))

In [1]:
########## Analyse ##########

print(allCar)

ModifyFile.traceCurve()

for vehicle in allCar:
    ModifyFile.calculationError(vehicle)

ModifyFile.calculationTot()

traci.close(True)

NameError: name 'listTotLenVeh' is not defined